# DL Assignment 5
## Sargun Singh (102115078) 4O1D

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [3]:
# Generate a synthetic binary classification dataset
X, y = make_classification(
    n_samples=1000,     # Number of samples
    n_features=20,      # Number of features
    n_informative=15,   # Number of informative features
    n_redundant=5,      # Number of redundant features
    n_classes=2,        # Number of classes
    random_state=42     # Seed for reproducibility
)

In [4]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Standardize the features for better performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
# Standardize the features for better performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [8]:
# Define a Deep Multi-Layer Neural Network
class DeepNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(DeepNN, self).__init__()
        self.hidden_layers = nn.ModuleList()
        in_features = input_size
        
        # Add hidden layers
        for hidden_size in hidden_sizes:
            self.hidden_layers.append(nn.Linear(in_features, hidden_size))
            in_features = hidden_size
        
        # Output layer
        self.output_layer = nn.Linear(in_features, output_size)
    
    def forward(self, x):
        for layer in self.hidden_layers:
            x = torch.relu(layer(x))
        x = self.output_layer(x)  # No activation for the output layer (handled by loss function)
        return x

In [9]:
# Model parameters
input_size = X_train.shape[1]  # Number of features
hidden_sizes = [64, 32, 16]    # Sizes of hidden layers
output_size = 2                # Number of classes

In [10]:
# Instantiate the model
model = DeepNN(input_size, hidden_sizes, output_size)

In [13]:
# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()  # Suitable for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.01)  # Adam optimizer

In [14]:
#Train the Model
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Print loss for every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 0.0349
Epoch [20/100], Loss: 0.0032
Epoch [30/100], Loss: 0.0002
Epoch [40/100], Loss: 0.0002
Epoch [50/100], Loss: 0.0001
Epoch [60/100], Loss: 0.0001
Epoch [70/100], Loss: 0.0000
Epoch [80/100], Loss: 0.0000
Epoch [90/100], Loss: 0.0000
Epoch [100/100], Loss: 0.0000


In [15]:
# Evaluate the Model
with torch.no_grad():
    # Make predictions
    train_predictions = torch.argmax(model(X_train_tensor), dim=1)
    test_predictions = torch.argmax(model(X_test_tensor), dim=1)
    
    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, train_predictions.numpy())
    test_accuracy = accuracy_score(y_test, test_predictions.numpy())

    print(f'Training Accuracy: {train_accuracy:.4f}')
    print(f'Testing Accuracy: {test_accuracy:.4f}')

Training Accuracy: 1.0000
Testing Accuracy: 0.9600
